In [2]:
#!pip install -U aeon
#!pip install aeon[all_extras]
import warnings
import pandas as pd
from matplotlib import pyplot as plt
from aeon.visualisation import plot_series
from all_functions import *
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb 
from sklearn.model_selection import GridSearchCV
from aeon.transformations.detrend import STLTransformer
import ast
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error, make_scorer
import os
from sklearn.linear_model import Ridge, RidgeCV
from datetime import datetime
from distutils.util import strtobool

warnings.filterwarnings("ignore")
%matplotlib inline
def convert_to_list(series_str):
    return eval(series_str)

def get_preds_hybrid(path, test_date, start_index):
    df = pd.read_csv(path, sep=";")
    results = {}
    filtered_df = df[df['test_range'] == test_date]
    columns_p1_to_p12 = filtered_df.loc[:, 'P1':'P12']
    values_list = columns_p1_to_p12.values.flatten().tolist()     
    results = pd.Series(values_list, index=start_index)
    return results

def generate_index(start_date, end_date):
    end_date_dt = pd.to_datetime(end_date)
    
    start_date_dt = pd.to_datetime(start_date)
    
    index = pd.period_range(start=start_date_dt, end=end_date_dt, freq='M')

    return index

def get_each_point_error(path, val_date):
    df = pd.read_csv(path, sep=";")
    
    df_filtered = df[df['test_range'] <= val_date].copy()

    df_filtered = df_filtered.iloc[::-1]
    
    error_series_concatenated = []
    for _, row in df_filtered.iterrows():
        error_series_str = row['error_series']
        error_list = ast.literal_eval(error_series_str)
        error_series_concatenated.append(error_list)
    return error_series_concatenated

/home/user/miniconda3/envs/lucas/lib/python3.11/site-packages/rapids_dask_dependency/dask_loader.py:36: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  mod = importlib.import_module(spec.name)


In [3]:
# models = ["ETS","catboost", "arima", "rf"]
# transformations = ["normal", "deseasonal"]
from dtaidistance import dtw
from scipy.stats import norm
from tslearn.barycenters import \
    euclidean_barycenter, \
    dtw_barycenter_averaging, \
    dtw_barycenter_averaging_subgradient, \
    softdtw_barycenter

from aeon.clustering.averaging import elastic_barycenter_average
from minisom import MiniSom
dirs = [
    '../datasets/venda/mensal/uf/gasolinac/',
    '../datasets/venda/mensal/uf/etanolhidratado/',
    '../datasets/venda/mensal/uf/glp/',
    '../datasets/venda/mensal/uf/oleodiesel/',
    '../datasets/venda/mensal/uf/querosenedeaviacao/',
]
horizon = 12
window = 12
model_transformation = [
                        "GASF_ridge_deseasonal",
                        "GADF_ridge_deseasonal",
                        "STFT_ridge_deseasonal",
                        "MTF_ridge_deseasonal",
                        "RP_ridge_deseasonal",
                        "FIRTS_ridge_deseasonal",
                        "DWT_ridge_deseasonal",
                        "SWT_ridge_deseasonal",
                        "CWT_ridge_deseasonal",
                        "ETS_normal",
                        "WPT_ridge_deseasonal",
                        "catboost_deseasonal",
                        "ridge_deseasonal",
                        "arima_deseasonal"
                        ]
dates = [
        # '1993-03_1994-02',
        # '1994-03_1995-02', '1995-03_1996-02', '1996-03_1997-02', '1997-03_1998-02', '1998-03_1999-02',
        # '1999-03_2000-02', '2000-03_2001-02', '2001-03_2002-02', '2002-03_2003-02', '2003-03_2004-02',
        # '2004-03_2005-02', '2005-03_2006-02', '2006-03_2007-02', '2007-03_2008-02', '2008-03_2009-02',
        # '2009-03_2010-02', '2010-03_2011-02', '2011-03_2012-02', '2012-03_2013-02', '2013-03_2014-02',
        # '2014-03_2015-02', '2015-03_2016-02', '2016-03_2017-02', '2017-03_2018-02', '2018-03_2019-02',
        '2019-03_2020-02', '2020-03_2021-02', '2021-03_2022-02', '2022-03_2023-02', '2023-03_2024-02',
         ]


date_past = {
            '2019-03_2020-02': '2018-03_2019-02',
            '2020-03_2021-02': '2019-03_2020-02',
            '2021-03_2022-02': '2020-03_2021-02',
            '2022-03_2023-02': '2021-03_2022-02',
            '2023-03_2024-02': '2022-03_2023-02',
             }

produto = "gasolinac"
estado = "SP"

combination_format = "som"
for directory in dirs:
    all_rmse = []
    all_pocid = []
    all_mape = []
    all_pbe = []
    for file in os.listdir(directory):
        if file.endswith('.csv'):
            uf = file.split("_")[1].upper()
            derivado = file.split("_")[2].split(".")[0]
            
            # if uf == estado and produto == derivado:
            full_path = os.path.join(directory, file)
            series = read_series(full_path)
            
            all_preds_dates = []
            all_test = []


            for date in dates:
                start_date, end_date = date.split('_')
                test_index = generate_index(start_date, end_date)
                test_real = get_test_real(series, start_date, end_date)
                row_data = []
                all_preds_year = []

                preds_barycenter = []
                preds_elastic = []
                
                for mt in model_transformation:
                    model, transform = mt.rsplit("_", 1)

                    path_result = f'./paper_roma/{model}/{derivado}/{transform}/transform_{uf}.csv'

                    serie_pred = get_preds_hybrid(path_result, date, test_index)

                    #com base no erro
                    # error_past = get_each_point_error(path_result, date_past[date])[0]
                    # mean_error = abs(sum(error_past) / len(error_past))
                    # mean_error = abs(np.mean(error_past))
                    # std_error = np.std(error_past)

                    # lower = [pred - mean_error for pred in serie_pred.tolist()]
                    # upper = [pred + mean_error for pred in serie_pred.tolist()]
                    # lower_mean = [pred - mean_error for pred in serie_pred]
                    # upper_mean = [pred + mean_error for pred in serie_pred]

                    # z = norm.ppf(0.95)
                    # lower = [pred - z * std_error for pred in serie_pred]
                    # upper = [pred + z * std_error for pred in serie_pred]


                    # lower_upper = [lower, upper]
                    # result_lower_upper = dtw_barycenter_averaging_subgradient(lower_upper, max_iter=35)
                    # result_lower_upper = result_lower_upper.flatten().tolist()
                    # preds_barycenter.append(result_lower_upper)
                    # all_preds_year.append(result_lower_upper)

                    # preds_barycenter.append(upper)
                    # preds_barycenter.append(lower)

                    # all_preds_year.append(upper)
                    # all_preds_year.append(lower)

                    # adiciona 10% para intervalo de confianca
                    # alpha = 0.1
                    # lower = [p - alpha * p for p in serie_pred.tolist()]
                    # upper = [p + alpha * p for p in serie_pred.tolist()]

                    # print(mt)
                    # print(upper)
                    # print(serie_pred.tolist())
                    # print(lower)
                    # print("--------")


                    
                    # print(serie_pred)
                    all_preds_year.append(serie_pred.tolist())
                    

                    preds_barycenter.append(serie_pred.tolist())
                    
                    preds_elastic.append(np.array([[serie_pred.tolist()]]))
                    
                    # preds_elastic.append(np.array([[lower]]))
                    # preds_elastic.append(np.array([[upper]]))
                    # preds_elastic.append(np.array([[result_lower_upper]]))
                    # preds_elastic.append(np.array([[serie_pred.values]]))
                
                all_test.extend(test_real)
                
                if combination_format == "mean":
                    mean_preds_year = np.mean(np.array(all_preds_year), axis=0)
                elif combination_format == "median":
                    mean_preds_year = np.median(np.array(all_preds_year), axis=0)
                elif combination_format == "softdtw":
                    #softdtw barycenter
                    softdtw_preds = softdtw_barycenter(preds_barycenter, max_iter=35, gamma=0.01)
                    mean_preds_year = softdtw_preds.flatten().tolist()
                elif combination_format == "dtw_subgradient":
                    #dtw avg barycenter subradient
                    preds_dtw_subgradient = dtw_barycenter_averaging_subgradient(preds_barycenter, max_iter=35, random_state=0)
                    mean_preds_year = preds_dtw_subgradient.flatten().tolist()

                elif combination_format == "dtw_subgradient2":
                    #dtw avg barycenter subradient
                    metric_params = {
                                        "global_constraint": "itakura",
                                        "sakoe_chiba_radius": 6, 
                                        "itakura_max_slope": 1.5
                                    }
                    
                    # init = np.median(np.array(all_preds_year), axis=0)
                    # init = init.reshape(1, -1).T
                    shape = 'normal'
                    init = None
                    X = np.array(preds_barycenter).reshape((len(preds_barycenter)), len(preds_barycenter[0]), 1)
                    
                    preds_dtw_subgradient = dtw_barycenter_averaging_subgradient(X, init_barycenter=init, max_iter=35, random_state=0, metric_params=metric_params)
                    mean_preds_year = preds_dtw_subgradient.flatten().tolist()


                elif combination_format == "dtw":
                    #dtw barycenter averaging
                    preds_dtw_avg = dtw_barycenter_averaging(preds_barycenter, max_iter=35)
                    mean_preds_year = preds_dtw_avg.flatten().tolist()
                
                elif combination_format == "eba":
                    #elastic barycenter average
                    capt = np.vstack(preds_elastic)
                    shape = "twe"
                    # X = np.array(preds_elastic).reshape(len(preds_elastic), 1, 12)
                    # init = np.median(np.array(all_preds_year), axis=0)
                    # init = init.reshape(1, -1)
                    init = 'random'
                    elastic = elastic_barycenter_average(capt, distance=shape, init_barycenter=init, random_state=0),
                    mean_preds_year = elastic[0].tolist()[0]
                elif combination_format == "som":
                    all_preds_year = np.array(all_preds_year)
                    shape = 'mean'
                    som_width, som_height = 2, 2
                    som = MiniSom(som_width, som_height, input_len=all_preds_year.shape[1], sigma=0.8, learning_rate=0.3, random_seed=0)
                    som.random_weights_init(all_preds_year)
                    som.train_random(all_preds_year, 200)
                    bmu_indices = np.array([som.winner(x) for x in all_preds_year])

                    final_prediction = []
                    for i in range(all_preds_year.shape[1]):
                        cluster_means = []
                        for x, y in np.unique(bmu_indices, axis=0):
                            cluster_data = all_preds_year[(bmu_indices[:, 0] == x) & (bmu_indices[:, 1] == y), i]
                            if len(cluster_data) > 0:
                                cluster_means.append(cluster_data.mean())
                                print(cluster_data)
                        final_prediction.append(np.mean(cluster_means)) 
                    # mean_preds_year = final_prediction
                all_preds_dates.extend(mean_preds_year)

            rmse_result = rmse(all_test, all_preds_dates)
            mape_result = mape(all_test, all_preds_dates)
            pocid_result = pocid(all_test, all_preds_dates)
            pbe_result = pbe(pd.Series(all_test), pd.Series(all_preds_dates))
            mcpm_result = mcpm(rmse_result, mape_result, pocid_result)

            all_pocid.append(pocid_result)
            all_pbe.append(pbe_result)
            all_rmse.append(rmse_result)
            all_mape.append(mape_result)
        
    

    if len(all_rmse) > 0:
        media_rmse = sum(all_rmse) / len(all_rmse)
        media_mape = sum(all_mape) / len(all_mape)
        media_pocid = sum(all_pocid) / len(all_pocid)
        media_pbe = sum(all_pbe) / len(all_pbe)

        representacao = f'{combination_format}_{shape}'

        data = {
            "representacao": [representacao],
            "RMSE": [media_rmse],
            "MAPE": [media_mape],
            "POCID": [media_pocid],
            "PBE": [media_pbe],
            "derivado": [derivado]
        }

        df = pd.DataFrame(data)

        csv_file = f'./metrics2/{derivado}.csv'
        os.makedirs('./metrics2/', exist_ok=True)

        df.to_csv(csv_file, mode='a', header=not pd.io.common.file_exists(csv_file), index=False)


[95083.29635088 95975.46774454 94948.44731053 94904.82073359]
[95630.60624758 95510.20676352 95879.93712835 95798.8512155
 95896.2895904  95905.00814599]
[95107.51605042 95905.00814599]
[92719.76690981 95256.47512741]
[94380.82249293 94903.30097995 94764.1018085  95090.09232506]
[91973.9639792  92396.92527536 91848.91721162 92039.51011661
 91728.30037531 92068.30326817]
[94118.2421303  94111.40845078]
[91962.26220054 91453.24959009]
[95739.29218841 96007.9846254  94302.06680264 96423.28487189]
[93699.80083637 93141.03720674 93566.08043297 93432.24709444
 93359.52131989 93625.51643664]
[94606.09345668 96018.57971621]
[92225.13507427 94483.48093897]
[95464.64462365 96045.33189272 95316.14478465 94709.85188788]
[92705.13892762 92669.25710575 93034.75324912 93100.98330763
 92867.78595674 93214.75999989]
[94570.48838742 95265.74137258]
[93358.30363319 92114.16201392]
[ 99042.51417315  99826.4628604   99099.34142092 100991.24494543]
[97750.63372578 97923.16729785 97777.72897954 97823.6621418

NameError: name 'mean_preds_year' is not defined